<a href="https://colab.research.google.com/github/Daniel-hp/LSTM/blob/master/LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np


with open('salida_300_propio.txt') as f:
    data = f.readlines()

print("Salida ")
print(data[-150:])





Salida 
['35/60 [================>.............] - ETA: 8:53 - loss: 0.0018 - acc: 0.9995\n', '36/60 [=================>............] - ETA: 8:31 - loss: 0.0018 - acc: 0.9995\n', '37/60 [=================>............] - ETA: 8:10 - loss: 0.0018 - acc: 0.9995\n', '38/60 [==================>...........] - ETA: 7:48 - loss: 0.0018 - acc: 0.9995\n', '39/60 [==================>...........] - ETA: 7:26 - loss: 0.0017 - acc: 0.9996\n', '40/60 [===================>..........] - ETA: 7:05 - loss: 0.0017 - acc: 0.9996\n', '41/60 [===================>..........] - ETA: 6:43 - loss: 0.0017 - acc: 0.9996\n', '42/60 [====================>.........] - ETA: 6:22 - loss: 0.0017 - acc: 0.9996\n', '43/60 [====================>.........] - ETA: 6:00 - loss: 0.0017 - acc: 0.9996\n', '44/60 [=====================>........] - ETA: 5:39 - loss: 0.0017 - acc: 0.9996\n', '45/60 [=====================>........] - ETA: 5:17 - loss: 0.0017 - acc: 0.9996\n', '46/60 [======================>.......] - ETA: 4:56 - lo

In [0]:
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, Flatten, Reshape, Softmax
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, ConvLSTM2D, TimeDistributed
from keras.utils import Sequence
from keras.optimizers import Adam, SGD
def normaliza(mayor, minimo, elem):
    if mayor == 0:
        return 0
    if (((elem - minimo) / (mayor - minimo)) * 10) > 10:
        return 10
    return ((elem - minimo) / (mayor - minimo)) * 10


def procesaEntrada(arr, h1_m=101, w1_m=199, h2_m=199, w2_m=199):
    #posiciones = []
    if len(arr) > 0:
        h1 = arr[:, 2]
        w1 = arr[:, 3]
        h2 = arr[:, 4]
        w2 = arr[:, 5]


        nv_h1 = copy.deepcopy(h1)
        nv_w1 = copy.deepcopy(w1)
        nv_h2 = copy.deepcopy(h2)
        nv_w2 = copy.deepcopy(w2)
        idx = 0
        for a, b, c, d in zip(w1, h1, w2, h2):
        #    if len(w1) < 5 :
        #        posiciones.append(idx)
        #        continue
            nv_h1[idx] = normaliza(h1_m, 0, b)
            nv_w1[idx] = normaliza(w1_m, 0, a)
            nv_h2[idx] = normaliza(h2_m, 0, d)
            nv_w2[idx] = normaliza(w2_m, 0, c)
            idx += 1
        return np.column_stack((np.array(arr[:, 0]), np.array(arr[:, 1]), np.array(nv_h1), np.array(nv_w1),
                                np.array(nv_h2), np.array(nv_w2)))
    else:
        return []

def convierteATexto(texto):
    cadena = np.array(np.zeros((texto.shape[0])), dtype = 'object')
    for m in range(0, len(cadena)):
        cadena[m] = [0]
    elim = 0
    posiciones = []
    for secuencia, num_sec in zip(texto, range(0,cadena.shape[0])):
        validador = len(secuencia)
        idx = 1
        if len(secuencia) < 5:
            elim += 1
            posiciones.append([num_sec])
            continue
        for elem in secuencia:
            cadena[num_sec] = np.append(cadena[num_sec],  elem)
            if idx < validador:
                cadena[num_sec] = np.append(cadena[num_sec],  [-1])
            idx += 1
    cadena = cadena[:len(cadena)-elim]
    return cadena, posiciones
if __name__ == '__main__':
    pckl = pd.read_pickle("conjunto_total.pckl")

    tag_to_indx = {"Agua": 1, "Alto": 2, "Apagar": 3, "Ayudar": 4, "Campo": 5, "Crecer": 6, "Cuerpo": 7, "Despedir": 8,
                   "Dinero": 9, "Escribir": 10, "Foto": 11, "Fruto": 12, "Hacer": 13, "LavarCara": 14, "Madera": 15,
                   "Miedo": 16, "Morder": 17, "Nariz": 18, "Padrino": 19, "Película": 20, "Persona": 21, "Primero": 22,
                   "Quedar": 23, "Saber": 24, "Sangre": 25, "Señorita": 26, "Suerte": 27, "Toro": 28}
    indx_to_tag = {1: "Agua", 2: "Alto", 3: "Apagar", 4: "Ayudar", 5: "Campo", 6: "Crecer", 7: "Cuerpo", 8: "Despedir",
                   9: "Dinero", 10: "Escribir", 11: "Foto", 12: "Fruto", 13: "Hacer", 14: "LavarCara", 15: "Madera",
                   16: "Miedo", 17: "Morder", 18: "Nariz", 19: "Padrino", 20: "Película", 21: "Persona", 22: "Primero",
                   23: "Quedar", 24: "Saber", 25: "Sangre", 26: "Señorita", 27: "Suerte", 28: "Toro"}
    X = pckl.iloc[:, 0].values




    max_h1, max_w1, max_h2, max_w2 = 0, 0, 0, 0
    for p in X:
        for m in p:
            if m[2] > max_h1:
                max_h1 = m[2]
            if m[3] > max_w1:
                max_w1 = m[3]
            if m[4] > max_h2:
                max_h2 = m[4]
            if m[5] > max_w2:
                max_w2 = m[5]


    X = np.array([procesaEntrada(l, max_h1, max_w1, max_h2, max_w2) for l in X])#convierteATexto()
    y = pckl.iloc[:, 1].values
    
    # Se procesa la entrada 
    y = [tag_to_indx[x]-1 for x in y]
    y = to_categorical(y)
    X = [np.array(l) for l in X]
    MAX_LENGTH = max([len(i)  for i in X])


    # Se hacen a todos los elementos de la entrada del mismo tamaño
    arreglo_total = pad_sequences(maxlen=MAX_LENGTH, sequences=X, padding="post")


    # Se divide el conjunto total en conjuntos de entrenamiento, prueba y dev
    X_train, X_test, y_train, y_test = train_test_split(arreglo_total, y, test_size=0.2)
    X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1)
    np.save("trainX.npy", X_train)
    np.save("trainY.npy", y_train)
    np.save("devX.npy", X_dev)
    np.save("devY.npy", y_dev)
    np.save("testX.npy",  X_test)
    np.save("testY.npy", y_test)
    

    # Se define al modelo
    model = Sequential()   
    trainX = X_train.reshape((X_train.shape[0], X_train.shape[1] *  X_train.shape[2]))
    devX = X_dev.reshape((X_dev.shape[0], X_dev.shape[1] *  X_dev.shape[2]))
    tam_input = 60
    model.add(LSTM(300, return_sequences=True, input_shape=(tam_input, trainX.shape[1])))
    model.add(LSTM(300, input_shape=(tam_input, trainX.shape[1])))
    model.add(Dropout(0.70))
    #model.add(Flatten())
    model.add(Dense(28, activation='softmax'))
    opt = Adam(lr = 0.001, clipnorm=.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    epochs = 1000
    batch_size = 2500


    # Se definen generadores 
    generator = TimeseriesGenerator(trainX, y_train, length=tam_input, batch_size=batch_size)
    generator2 = TimeseriesGenerator(devX, y_dev, length=tam_input, batch_size=batch_size)

    # Se entrena al modelo
    H = model.fit_generator(generator,validation_data=generator2,epochs=epochs, steps_per_epoch=tam_input, callbacks=[EarlyStopping(monitor='acc', patience=10, min_delta=0.0001)])
  
    model.save('modelo_trescientas_unidades_conjunto_propio.h5')
    """


In [13]:
import keras as keras
from keras.models import load_model
import numpy as np
from keras.preprocessing.sequence import TimeseriesGenerator
model = load_model('modelo_trescientas_unidades_conjunto_propio (1).h5')
print(model.summary())
X = np.load('trainX.npy')
Y = np.load('trainY.npy')
testX = X.reshape((X.shape[0], X.shape[1] *  X.shape[2]))
batch_size = 2500
tam_input = 60
nb_validation_samples = 100
generator = TimeseriesGenerator(testX, Y, length=tam_input, batch_size=batch_size)                                                                
score = model.evaluate_generator(generator, nb_validation_samples/batch_size)
scores = model.predict_generator(generator, nb_validation_samples/batch_size)

print(score)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 300)           8331600   
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                8428      
Total params: 9,061,228
Trainable params: 9,061,228
Non-trainable params: 0
_________________________________________________________________
None
[0.021495221182703972, 0.9986666440963745]


In [14]:

X = np.load('testX.npy')
Y = np.load('testY.npy')
testX = X.reshape((X.shape[0], X.shape[1] *  X.shape[2]))
batch_size = 2500
tam_input = 60
nb_validation_samples = 100
generator = TimeseriesGenerator(testX, Y, length=tam_input, batch_size=batch_size)                                                                
score = model.evaluate_generator(generator, nb_validation_samples/batch_size)
scores = model.predict_generator(generator, nb_validation_samples/batch_size)

print(score)

[9.803570747375488, 0.0181818176060915]
